In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.statespace.structural import UnobservedComponents
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from transformers import TimeSeriesTransformerForPrediction
import pmdarima as pm
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def prepare_data():
    # Population forecast (example data)
    years = np.arange(2022, 2071)
    population = np.linspace(1e6, 1.5e6, len(years))  # Replace with your actual data
    
    # Patient data (example - replace with your actual 2022-2023 data)
    patient_data = {
        2022: np.random.normal(50000, 5000, 12).cumsum(),  # Monthly data
        2023: np.random.normal(55000, 5500, 12).cumsum()   # Monthly data
    }
    
    # Create DataFrame
    pop_df = pd.DataFrame({'year': years, 'population': population})
    
    # Create monthly patient DataFrame
    patient_rows = []
    for year, months in patient_data.items():
        for month, patients in enumerate(months, 1):
            patient_rows.append({
                'year': year,
                'month': month,
                'patients': patients
            })
    patient_df = pd.DataFrame(patient_rows)
    
    # Aggregate to yearly for some models
    yearly_patient_df = patient_df.groupby('year')['patients'].sum().reset_index()
    
    return pop_df, patient_df, yearly_patient_df

def create_features(df, population_df):
    # Merge with population data
    df = df.merge(population_df, on='year', how='left')
    
    # Time features
    df['time_index'] = np.arange(len(df))
    df['year_sin'] = np.sin(2 * np.pi * df['time_index'] / 12)
    df['year_cos'] = np.cos(2 * np.pi * df['time_index'] / 12)
    
    # Lag features
    df['patients_lag1'] = df['patients'].shift(1)
    df['patients_lag12'] = df['patients'].shift(12)
    
    # Rolling statistics
    df['rolling_mean_12'] = df['patients'].rolling(window=12).mean()
    df['rolling_std_12'] = df['patients'].rolling(window=12).std()
    
    # Population ratio
    df['patients_per_capita'] = df['patients'] / df['population']
    
    return df.dropna()

In [ ]:
def classical_time_series_models(train, test):
    # ARIMA
    print("Fitting ARIMA...")
    arima_model = pm.auto_arima(train['patients'], seasonal=True, m=12,
                               suppress_warnings=True, stepwise=True)
    arima_pred = arima_model.predict(n_periods=len(test))
    
    # Exponential Smoothing
    print("Fitting Exponential Smoothing...")
    ets_model = ExponentialSmoothing(train['patients'], seasonal='add', seasonal_periods=12).fit()
    ets_pred = ets_model.forecast(len(test))
    
    # Bayesian Structural Time Series
    print("Fitting BSTS...")
    bsts_model = UnobservedComponents(train['patients'], level='local linear trend', seasonal=12)
    bsts_fit = bsts_model.fit()
    bsts_pred = bsts_fit.forecast(len(test))
    
    return {
        'ARIMA': arima_pred,
        'ETS': ets_pred,
        'BSTS': bsts_pred
    }


In [ ]:
def ml_models(X_train, y_train, X_test, scaler):
    # XGBoost
    print("Training XGBoost...")
    xgb = XGBRegressor(n_estimators=1000, learning_rate=0.01, early_stopping_rounds=50)
    xgb.fit(X_train, y_train, eval_set=[(X_train, y_train)], verbose=False)
    xgb_pred = xgb.predict(X_test)
    
    # Random Forest
    print("Training Random Forest...")
    rf = RandomForestRegressor(n_estimators=500, random_state=42)
    rf.fit(X_train, y_train)
    rf_pred = rf.predict(X_test)
    
    # Inverse transform if needed
    if scaler:
        xgb_pred = scaler.inverse_transform(xgb_pred.reshape(-1, 1)).flatten()
        rf_pred = scaler.inverse_transform(rf_pred.reshape(-1, 1)).flatten()
    
    return {
        'XGBoost': xgb_pred,
        'RandomForest': rf_pred
    }

def lstm_model(X_train, y_train, X_test, n_steps=12):
    print("Training LSTM...")
    # Reshape for LSTM [samples, timesteps, features]
    X_train = X_train.reshape((X_train.shape[0], n_steps, X_train.shape[1]//n_steps))
    X_test = X_test.reshape((X_test.shape[0], n_steps, X_test.shape[1]//n_steps))
    
    model = Sequential([
        LSTM(100, activation='relu', input_shape=(n_steps, X_train.shape[2])),
        Dropout(0.2),
        Dense(50, activation='relu'),
        Dense(1)
    ])
    model.compile(optimizer='adam', loss='mse')
    
    model.fit(X_train, y_train, epochs=100, verbose=0)
    
    lstm_pred = model.predict(X_test).flatten()
    return lstm_pred

# Transformer Model (requires PyTorch)
def transformer_model(train, test):
    print("Training Transformer...")
    # This is a simplified version - actual implementation would be more complex
    config = TimeSeriesTransformerForPrediction.config_class(
        prediction_length=len(test),
        context_length=len(train),
        input_size=1,
        num_time_features=1
    )
    model = TimeSeriesTransformerForPrediction(config)
    
    # Training would require more setup
    # For now we'll return naive forecast as placeholder
    transformer_pred = np.array([train[-1]] * len(test))
    return transformer_pred

# Ensemble Model
def ensemble_predictions(predictions, weights=None):
    if not weights:
        weights = np.ones(len(predictions)) / len(predictions)
    
    all_preds = np.array(list(predictions.values()))
    return np.average(all_preds, axis=0, weights=weights)

# Main forecasting function
def forecast_patients(pop_df, patient_df, forecast_years=5):
    # Prepare data
    monthly_data = patient_df.set_index(['year', 'month'])['patients'].unstack()
    monthly_data = monthly_data.stack().reset_index(name='patients')
    monthly_data = monthly_data.merge(pop_df, on='year')
    
    # Create features
    full_data = create_features(monthly_data, pop_df)
    
    # Split data
    train_size = int(len(full_data) * 0.8)
    train, test = full_data.iloc[:train_size], full_data.iloc[train_size:]
    
    # Classical models
    classical_preds = classical_time_series_models(train['patients'], test['patients'])
    
    # Prepare ML data
    features = ['time_index', 'population', 'year_sin', 'year_cos', 
                'patients_lag1', 'patients_lag12', 'rolling_mean_12', 'rolling_std_12']
    target = 'patients'
    
    X_train, y_train = train[features], train[target]
    X_test, y_test = test[features], test[target]
    
    # Scale data
    scaler = MinMaxScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # ML models
    ml_preds = ml_models(X_train_scaled, y_train, X_test_scaled, scaler)
    
    # LSTM
    lstm_pred = lstm_model(X_train_scaled, y_train, X_test_scaled)
    ml_preds['LSTM'] = lstm_pred
    
    # Transformer (placeholder)
    transformer_pred = transformer_model(train['patients'].values, test['patients'].values)
    ml_preds['Transformer'] = transformer_pred
    
    # Combine all predictions
    all_predictions = {**classical_preds, **ml_preds}
    
    # Evaluate models
    for name, pred in all_predictions.items():
        mae = mean_absolute_error(y_test, pred[:len(y_test)])
        print(f"{name} MAE: {mae:.2f}")
    
    # Create ensemble
    best_models = ['XGBoost', 'LSTM', 'ETS']  # Select based on performance
    ensemble_pred = ensemble_predictions(
        {k: v for k, v in all_predictions.items() if k in best_models},
        weights=[0.4, 0.3, 0.3]  # Adjust based on model performance
    )
    
    # Future forecasting
    print("\nGenerating future forecasts...")
    last_data = full_data.iloc[-1]
    future_years = np.arange(full_data['year'].max() + 1, full_data['year'].max() + forecast_years + 1)
    
    future_predictions = []
    for year in future_years:
        # Create future row (simplified - would need proper feature updates)
        future_row = last_data.copy()
        future_row['year'] = year
        future_row['time_index'] += 12
        future_row['year_sin'] = np.sin(2 * np.pi * future_row['time_index'] / 12)
        future_row['year_cos'] = np.cos(2 * np.pi * future_row['time_index'] / 12)
        
        # Get population for this year
        future_row['population'] = pop_df[pop_df['year'] == year]['population'].values[0]
        
        # Update lags and rolling stats (would need proper implementation)
        future_row['patients_lag1'] = last_data['patients']
        future_row['patients_lag12'] = full_data.iloc[-12]['patients'] if len(full_data) >= 12 else np.nan
        
        # Predict using ensemble
        future_features = future_row[features].values.reshape(1, -1)
        future_scaled = scaler.transform(future_features)
        
        # Get prediction from each model
        future_preds = {
            'XGBoost': xgb.predict(future_scaled)[0],
            'LSTM': lstm_model(future_scaled.reshape(1, 12, -1))[0],
            'ETS': ets_model.forecast(1)[0]
        }
        ensemble_future = ensemble_predictions(future_preds, weights=[0.4, 0.3, 0.3])
        
        future_row['patients'] = ensemble_future
        future_predictions.append(future_row)
        last_data = future_row.copy()
    
    future_df = pd.DataFrame(future_predictions)
    
    # Visualize results
    plt.figure(figsize=(15, 7))
    plt.plot(full_data['time_index'], full_data['patients'], label='Historical')
    plt.plot(test['time_index'], ensemble_pred, label='Ensemble Forecast', linestyle='--')
    plt.plot(future_df['time_index'], future_df['patients'], label='Future Forecast', linestyle=':')
    plt.title('Patient Forecast')
    plt.xlabel('Time Index')
    plt.ylabel('Patients')
    plt.legend()
    plt.grid()
    plt.show()
    
    return {
        'historical': full_data,
        'test_predictions': ensemble_pred,
        'future_predictions': future_df,
        'model_performance': {name: mean_absolute_error(y_test, pred[:len(y_test)]) 
                             for name, pred in all_predictions.items()}
    }

# Execute the forecasting
pop_df, patient_df, yearly_patient_df = prepare_data()
results = forecast_patients(pop_df, patient_df, forecast_years=10)